## Part 0: Database Setup (PROVIDED - DO NOT MODIFY)

This cell creates the database and all necessary tables. **Run this first!**

In [52]:
# Import libraries
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text

# Database credentials
DB_NAME = "car_dealership_hw"
DB_USER = "student"
DB_PASSWORD = ""
DB_HOST = "localhost"
DB_PORT = "5432"

print("=" * 80)
print("STEP 1: Creating database...")

# Create database
try:
    conn = psycopg2.connect(dbname="postgres", user=DB_USER, password=DB_PASSWORD,
                           host=DB_HOST, port=DB_PORT)
    conn.autocommit = True
    cursor = conn.cursor()
    
    cursor.execute(f"""
        SELECT pg_terminate_backend(pg_stat_activity.pid)
        FROM pg_stat_activity
        WHERE pg_stat_activity.datname = '{DB_NAME}'
          AND pid <> pg_backend_pid();
    """)
    
    cursor.execute(f"DROP DATABASE IF EXISTS {DB_NAME}")
    cursor.execute(f"CREATE DATABASE {DB_NAME}")
    print(f"✓ Database '{DB_NAME}' created!")
    
    cursor.close()
    conn.close()
except Exception as e:
    print(f"❌ Error: {e}")

# Connect to new database
print("\nSTEP 2: Connecting to database...")
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)
print("✓ Connected!")

# Create all tables
print("\nSTEP 3: Creating tables...")
create_tables_sql = """
-- Table: make (car manufacturers)
CREATE TABLE make (
    make_id SERIAL PRIMARY KEY,
    make_name VARCHAR(50) NOT NULL UNIQUE
);

-- Table: model (car models)
CREATE TABLE model (
    model_id SERIAL PRIMARY KEY,
    model_name VARCHAR(50) NOT NULL UNIQUE
);

-- Table: color (available colors)
CREATE TABLE color (
    color_id SERIAL PRIMARY KEY,
    color_name VARCHAR(30) NOT NULL UNIQUE
);

-- Table: fuel_type
CREATE TABLE fuel_type (
    fuel_type_id SERIAL PRIMARY KEY,
    fuel_type_name VARCHAR(30) NOT NULL UNIQUE
);

-- Table: features
CREATE TABLE features (
    feature_id SERIAL PRIMARY KEY,
    feature_name VARCHAR(50) NOT NULL UNIQUE
);

-- Table: inventory (main table with foreign keys)
CREATE TABLE inventory (
    inventory_id SERIAL PRIMARY KEY,
    make_id INTEGER NOT NULL REFERENCES make(make_id),
    model_id INTEGER NOT NULL REFERENCES model(model_id),
    color_id INTEGER REFERENCES color(color_id),
    fuel_type_id INTEGER REFERENCES fuel_type(fuel_type_id),
    year INTEGER NOT NULL,
    price DECIMAL(10,2),
    in_stock BOOLEAN DEFAULT TRUE,
    mileage INTEGER DEFAULT 0
);

-- Table: inventory_features (many-to-many)
CREATE TABLE inventory_features (
    inventory_id INTEGER REFERENCES inventory(inventory_id) ON DELETE CASCADE,
    feature_id INTEGER REFERENCES features(feature_id),
    PRIMARY KEY (inventory_id, feature_id)
);
"""

with engine.connect() as conn:
    conn.execute(text(create_tables_sql))
    conn.commit()
    print("✓ All tables created!")

# Insert initial reference data
print("\nSTEP 4: Loading initial reference data...")
initial_data_sql = """
-- Insert car makes
INSERT INTO make (make_name) VALUES
    ('Toyota'), ('Honda'), ('Ford'), ('Chevrolet'), ('Tesla'),
    ('BMW'), ('Mercedes'), ('Audi'), ('Nissan'), ('Mazda'), ('Volkswagen');

-- Insert car models
INSERT INTO model (model_name) VALUES
    ('Camry'), ('RAV4'), ('Accord'), ('Mustang'), ('Corvette'),
    ('Model 3'), ('X5'), ('C-Class'), ('A4'), ('Altima'),
    ('CX-5'), ('F-150'), ('Jetta');

-- Insert colors
INSERT INTO color (color_name) VALUES
    ('Blue'), ('Silver'), ('White'), ('Red'), ('Yellow'),
    ('Black'), ('Gray'), ('Green');

-- Insert fuel types
INSERT INTO fuel_type (fuel_type_name) VALUES
    ('Gasoline'), ('Diesel'), ('Electric'), ('Hybrid');

-- Insert features
INSERT INTO features (feature_name) VALUES
    ('GPS Navigation'), ('Sunroof'), ('Leather Seats'), 
    ('Backup Camera'), ('Bluetooth'), ('Heated Seats'),
    ('All-Wheel Drive'), ('Lane Assist'), ('Cruise Control');
"""

with engine.connect() as conn:
    conn.execute(text(initial_data_sql))
    conn.commit()
    print("✓ Reference data loaded!")

print("\n" + "=" * 80)
print("DATABASE READY! You can now complete the homework exercises.")
print("=" * 80)

STEP 1: Creating database...
✓ Database 'car_dealership_hw' created!

STEP 2: Connecting to database...
✓ Connected!

STEP 3: Creating tables...
✓ All tables created!

STEP 4: Loading initial reference data...
✓ Reference data loaded!

DATABASE READY! You can now complete the homework exercises.


## Part 1: Exploring the Database Structure

Before modifying data, let's understand the database structure.

### Exercise 1.1: List all tables in the database

Write a query to show all table names.

**Hint:** Query `information_schema.tables` where `table_schema = 'public'`

In [53]:
# SOLUTION
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
"""

df = pd.read_sql(text(query), engine)
df

,table_name
0,color
1,features
2,fuel_type
3,inventory
4,inventory_features
5,make
6,model


### Exercise 1.2: View all car makes

Display all records from the `make` table.

In [54]:
# SOLUTION
query = """
SELECT * FROM make
ORDER BY make_name;
"""

df = pd.read_sql(text(query), engine)
df

,make_id,make_name
0,8,Audi
1,6,BMW
2,4,Chevrolet
3,3,Ford
4,2,Honda
5,10,Mazda
6,7,Mercedes
7,9,Nissan
8,5,Tesla
9,1,Toyota


### Exercise 1.3: View all fuel types

Display all records from the `fuel_type` table.

In [55]:
# SOLUTION
query = """
SELECT * FROM fuel_type
ORDER BY fuel_type_name;
"""

df = pd.read_sql(text(query), engine)
df

,fuel_type_id,fuel_type_name
0,2,Diesel
1,3,Electric
2,1,Gasoline
3,4,Hybrid


## Part 2: COPY - Bulk Loading from CSV

Load inventory data from a CSV file using the COPY command.

**CSV File:** `/workspaces/Fall2025-MS3083-Base_Template/data/car_dealership_inventory.csv`

### Exercise 2.1: Preview the CSV file structure

Before loading the CSV, let's understand what columns we need for the COPY command.

**Task:** View the column names and first few lines of the CSV file using a shell command.

In [56]:
# SOLUTION - Use shell command to preview CSV
import subprocess

csv_path = "/workspaces/Fall2025-MS3083-Base_Template/data/car_dealership_inventory.csv"

# Show first 5 lines of the file
result = subprocess.run(['head', '-n', '11', csv_path], capture_output=True, text=True)
print("First 10 rows of CSV file:")
print("=" * 80)
print(result.stdout)

First 10 rows of CSV file:
inventory_id,make_id,model_id,color_id,fuel_type_id,year,price,in_stock,mileage
1,1,1,1,1,2023,28500.00,TRUE,0
2,1,2,2,2,2022,42000.00,TRUE,1500
3,2,3,3,1,2023,32000.00,TRUE,0
4,3,4,4,1,2021,45000.00,TRUE,8500
5,4,5,5,1,2023,75000.00,FALSE,2000
6,5,6,1,3,2023,48000.00,TRUE,0
7,6,7,6,4,2022,65000.00,TRUE,3500
8,7,8,2,4,2023,52000.00,TRUE,1200
9,8,9,7,1,2022,41000.00,TRUE,5600
10,9,10,3,1,2023,38000.00,FALSE,0



### Exercise 2.2: Load CSV into inventory table using \copy

Use psql's `\copy` command to bulk load the CSV data into the `inventory` table.

**Note:** We use `\copy` (with backslash) instead of `COPY` because it's a client-side command that works without special database privileges.

In [57]:
# SOLUTION
import subprocess

csv_path = "/workspaces/Fall2025-MS3083-Base_Template/data/car_dealership_inventory.csv"

# Use psql's \copy command (client-side, works without special privileges)
# We pipe the command to psql via stdin
copy_command = f"\\copy inventory (inventory_id, make_id, model_id, color_id, fuel_type_id, year, price, in_stock, mileage) FROM '{csv_path}' WITH (FORMAT CSV, HEADER TRUE);"

# Execute using psql with command piped to stdin
result = subprocess.run(
    ['psql', '-d', 'car_dealership_hw', '-U', 'student'],
    input=copy_command,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✓ Loaded data from CSV into inventory table")
    print(result.stdout.strip())
else:
    print(f"Error: {result.stderr}")

# Fix the sequence to continue from the max inventory_id
# This is necessary because we loaded data with explicit IDs from the CSV
with engine.connect() as conn:
    # Get the max inventory_id
    max_id_query = "SELECT MAX(inventory_id) FROM inventory;"
    result = conn.execute(text(max_id_query))
    max_id = result.fetchone()[0]
    
    # Reset the sequence to continue from max_id + 1
    # Only reset sequence if data was loaded
    if max_id is not None:
        # Reset the sequence to continue from max_id + 1
        reset_seq_query = f"SELECT setval('inventory_inventory_id_seq', {max_id});"
        conn.execute(text(reset_seq_query))
        conn.commit()
        print(f"✓ Reset sequence to continue from ID {max_id + 1}")
    else:
        print("⚠ Warning: No data in inventory table. Check if CSV loaded correctly.")

# Verify the load
count_query = "SELECT COUNT(*) as total FROM inventory;"
df_count = pd.read_sql(text(count_query), engine)
print(f"\nTotal inventory records: {df_count['total'][0]}")

✓ Loaded data from CSV into inventory table
COPY 15
✓ Reset sequence to continue from ID 16

Total inventory records: 15
✓ Reset sequence to continue from ID 16

Total inventory records: 15


### Exercise 2.3: Verify loaded data with a JOIN query

Write a query that joins `inventory` with `make`, `model`, `color`, and `fuel_type` to show human-readable data.

**Expected columns:** inventory_id, make_name, model_name, color_name, fuel_type_name, year, price, in_stock

In [58]:
# SOLUTION
query = """
SELECT 
    i.inventory_id,
    m.make_name,
    mo.model_name,
    c.color_name,
    ft.fuel_type_name,
    i.year,
    i.price,
    i.in_stock,
    i.mileage
FROM inventory i
INNER JOIN make m ON i.make_id = m.make_id
INNER JOIN model mo ON i.model_id = mo.model_id
LEFT JOIN color c ON i.color_id = c.color_id
LEFT JOIN fuel_type ft ON i.fuel_type_id = ft.fuel_type_id
ORDER BY i.inventory_id;
"""

df = pd.read_sql(text(query), engine)
df

,inventory_id,make_name,model_name,color_name,fuel_type_name,year,price,in_stock,mileage
0,1,Toyota,Camry,Blue,Gasoline,2023,28500.0,True,0
1,2,Toyota,RAV4,Silver,Diesel,2022,42000.0,True,1500
2,3,Honda,Accord,White,Gasoline,2023,32000.0,True,0
3,4,Ford,Mustang,Red,Gasoline,2021,45000.0,True,8500
4,5,Chevrolet,Corvette,Yellow,Gasoline,2023,75000.0,False,2000
5,6,Tesla,Model 3,Blue,Electric,2023,48000.0,True,0
6,7,BMW,X5,Black,Hybrid,2022,65000.0,True,3500
7,8,Mercedes,C-Class,Silver,Hybrid,2023,52000.0,True,1200
8,9,Audi,A4,Gray,Gasoline,2022,41000.0,True,5600
9,10,Nissan,Altima,White,Gasoline,2023,38000.0,False,0


## Part 3: INSERT - Adding New Data

Practice INSERT statements with the normalized database structure.

### Exercise 3.1: Insert new colors into the color table

Add the following colors to the `color` table:
- Orange
- Purple
- Brown

**Use a single INSERT statement with multiple values.**

In [59]:
# SOLUTION
query = """
INSERT INTO color (color_name)
VALUES 
    ('Orange'),
    ('Purple'),
    ('Brown');
"""

with engine.connect() as conn:
    result = conn.execute(text(query))
    conn.commit()
    print(f"✓ Inserted {result.rowcount} new color(s)")

# Verify
verify_query = "SELECT * FROM color ORDER BY color_id;"
df = pd.read_sql(text(verify_query), engine)
print("\nAll colors:")
df

✓ Inserted 3 new color(s)

All colors:


,color_id,color_name
0,1,Blue
1,2,Silver
2,3,White
3,4,Red
4,5,Yellow
5,6,Black
6,7,Gray
7,8,Green
8,9,Orange
9,10,Purple


### Exercise 3.2: Insert a new car into inventory

Add a new car with the following details:
- Make: Toyota (make_id = 1)
- Model: Camry (model_id = 1)
- Color: Orange (find the color_id you just created)
- Fuel Type: Hybrid (fuel_type_id = 4)
- Year: 2024
- Price: 35000.00
- In Stock: TRUE
- Mileage: 0

**First find the color_id for Orange, then insert the car.**

In [60]:
# SOLUTION - Step 1: Find color_id for Orange
find_color_query = "SELECT color_id FROM color WHERE color_name = 'Orange';"
df_color = pd.read_sql(text(find_color_query), engine)
orange_color_id = df_color['color_id'][0]
print(f"Orange color_id: {orange_color_id}")

# Step 2: Insert the car
insert_query = f"""
INSERT INTO inventory (make_id, model_id, color_id, fuel_type_id, year, price, in_stock, mileage)
VALUES (1, 1, {orange_color_id}, 4, 2024, 35000.00, TRUE, 0)
RETURNING inventory_id, year, price;
"""

with engine.connect() as conn:
    result = conn.execute(text(insert_query))
    returned = result.fetchone()
    conn.commit()
    print(f"\n✓ Inserted new car!")
    print(f"  Inventory ID: {returned[0]}")
    print(f"  Year: {returned[1]}")
    print(f"  Price: ${returned[2]}")

Orange color_id: 9

✓ Inserted new car!
  Inventory ID: 16
  Year: 2024
  Price: $35000.00


### Exercise 3.3: Insert multiple cars at once

Add these 3 cars to the inventory in a single INSERT statement:

1. Honda Accord, Blue, Gasoline, 2024, $34000, In Stock, 0 miles
2. Ford Mustang, Red, Gasoline, 2023, $48000, In Stock, 500 miles
3. Tesla Model 3, White, Electric, 2024, $52000, TRUE, 0 miles

**Hint:** Use the reference data IDs already in the database.

In [61]:
# SOLUTION
query = """
INSERT INTO inventory (make_id, model_id, color_id, fuel_type_id, year, price, in_stock, mileage)
VALUES 
    (2, 3, 1, 1, 2024, 34000.00, TRUE, 0),      -- Honda Accord, Blue, Gasoline
    (3, 4, 4, 1, 2023, 48000.00, TRUE, 500),    -- Ford Mustang, Red, Gasoline
    (5, 6, 3, 3, 2024, 52000.00, TRUE, 0);      -- Tesla Model 3, White, Electric
"""

with engine.connect() as conn:
    result = conn.execute(text(query))
    conn.commit()
    print(f"✓ Inserted {result.rowcount} new car(s)")

# Verify with JOIN
verify_query = """
SELECT i.inventory_id, m.make_name, mo.model_name, c.color_name, ft.fuel_type_name, i.year, i.price
FROM inventory i
INNER JOIN make m ON i.make_id = m.make_id
INNER JOIN model mo ON i.model_id = mo.model_id
INNER JOIN color c ON i.color_id = c.color_id
INNER JOIN fuel_type ft ON i.fuel_type_id = ft.fuel_type_id
ORDER BY i.inventory_id DESC
LIMIT 3;
"""

df = pd.read_sql(text(verify_query), engine)
print("\nNewly inserted cars:")
df

✓ Inserted 3 new car(s)

Newly inserted cars:


,inventory_id,make_name,model_name,color_name,fuel_type_name,year,price
0,19,Tesla,Model 3,White,Electric,2024,52000.0
1,18,Ford,Mustang,Red,Gasoline,2023,48000.0
2,17,Honda,Accord,Blue,Gasoline,2024,34000.0


## Part 4: UPDATE - Modifying Data

Practice UPDATE statements to modify existing records.

### Exercise 4.1: Update fuel type names to lowercase

Convert all `fuel_type_name` values in the `fuel_type` table to lowercase.

**Example:** 'Gasoline' → 'gasoline', 'Electric' → 'electric'

**Use the LOWER() function.**

In [62]:
# SOLUTION - First show current values
print("BEFORE UPDATE:")
df_before = pd.read_sql(text("SELECT * FROM fuel_type ORDER BY fuel_type_id;"), engine)
print(df_before)

# Update to lowercase
query = """
UPDATE fuel_type
SET fuel_type_name = LOWER(fuel_type_name);
"""

with engine.connect() as conn:
    result = conn.execute(text(query))
    conn.commit()
    print(f"\n✓ Updated {result.rowcount} fuel type(s) to lowercase")

# Show updated values
print("\nAFTER UPDATE:")
df_after = pd.read_sql(text("SELECT * FROM fuel_type ORDER BY fuel_type_id;"), engine)
df_after

BEFORE UPDATE:
   fuel_type_id fuel_type_name
0             1       Gasoline
1             2         Diesel
2             3       Electric
3             4         Hybrid

✓ Updated 4 fuel type(s) to lowercase

AFTER UPDATE:


,fuel_type_id,fuel_type_name
0,1,gasoline
1,2,diesel
2,3,electric
3,4,hybrid


### Exercise 4.2: Convert make and model names to lowercase

Update both the `make` and `model` tables to convert all names to lowercase.

**Write TWO separate UPDATE statements.**

In [63]:
# SOLUTION
print("BEFORE UPDATE:")
df_makes = pd.read_sql(text("SELECT * FROM make ORDER BY make_id LIMIT 5;"), engine)
df_models = pd.read_sql(text("SELECT * FROM model ORDER BY model_id LIMIT 5;"), engine)
print("Makes:")
print(df_makes)
print("\nModels:")
print(df_models)

# Update make table
query_make = """
UPDATE make
SET make_name = LOWER(make_name);
"""

# Update model table
query_model = """
UPDATE model
SET model_name = LOWER(model_name);
"""

with engine.connect() as conn:
    result1 = conn.execute(text(query_make))
    result2 = conn.execute(text(query_model))
    conn.commit()
    print(f"\n✓ Updated {result1.rowcount} make(s) to lowercase")
    print(f"✓ Updated {result2.rowcount} model(s) to lowercase")

# Show updated values
print("\nAFTER UPDATE:")
df_makes_after = pd.read_sql(text("SELECT * FROM make ORDER BY make_id LIMIT 5;"), engine)
df_models_after = pd.read_sql(text("SELECT * FROM model ORDER BY model_id LIMIT 5;"), engine)
print("Makes:")
print(df_makes_after)
print("\nModels:")
print(df_models_after)

BEFORE UPDATE:
Makes:
   make_id  make_name
0        1     Toyota
1        2      Honda
2        3       Ford
3        4  Chevrolet
4        5      Tesla

Models:
   model_id model_name
0         1      Camry
1         2       RAV4
2         3     Accord
3         4    Mustang
4         5   Corvette

✓ Updated 11 make(s) to lowercase
✓ Updated 13 model(s) to lowercase

AFTER UPDATE:
Makes:
   make_id  make_name
0        1     toyota
1        2      honda
2        3       ford
3        4  chevrolet
4        5      tesla

Models:
   model_id model_name
0         1      camry
1         2       rav4
2         3     accord
3         4    mustang
4         5   corvette


### Exercise 4.3: Apply discount to high-mileage cars

Reduce the price by 15% for all cars in the `inventory` table that have:
- Mileage > 5000
- Are currently in stock

**Use ROUND() to round to 2 decimal places.**

In [64]:
# SOLUTION - First preview affected cars
print("Cars that will be discounted:")
preview_query = """
SELECT i.inventory_id, m.make_name, mo.model_name, i.mileage, i.price, i.in_stock
FROM inventory i
INNER JOIN make m ON i.make_id = m.make_id
INNER JOIN model mo ON i.model_id = mo.model_id
WHERE i.mileage > 5000 AND i.in_stock = TRUE;
"""
df_preview = pd.read_sql(text(preview_query), engine)
print(df_preview)

# Apply discount
update_query = """
UPDATE inventory
SET price = ROUND(price * 0.85, 2)
WHERE mileage > 5000 AND in_stock = TRUE;
"""

with engine.connect() as conn:
    result = conn.execute(text(update_query))
    conn.commit()
    print(f"\n✓ Applied 15% discount to {result.rowcount} car(s)")

# Show updated prices
print("\nUpdated prices:")
df_after = pd.read_sql(text(preview_query), engine)
df_after

Cars that will be discounted:
   inventory_id   make_name model_name  mileage    price  in_stock
0             4        ford    mustang     8500  45000.0      True
1             9        audi         a4     5600  41000.0      True
2            11       mazda       cx-5     7200  34000.0      True
3            12      toyota      camry    12000  24500.0      True
4            13       honda     accord     6800  29500.0      True
5            15  volkswagen      jetta     9500  27500.0      True

✓ Applied 15% discount to 6 car(s)

Updated prices:


,inventory_id,make_name,model_name,mileage,price,in_stock
0,4,ford,mustang,8500,38250.0,True
1,9,audi,a4,5600,34850.0,True
2,11,mazda,cx-5,7200,28900.0,True
3,12,toyota,camry,12000,20825.0,True
4,13,honda,accord,6800,25075.0,True
5,15,volkswagen,jetta,9500,23375.0,True


### Exercise 4.4: Mark cars as out of stock

Update the `inventory` table to set `in_stock = FALSE` for all cars with year = 2021.

**First SELECT to see which cars will be affected, then UPDATE.**

In [65]:
# SOLUTION - Preview affected cars
print("Cars that will be marked out of stock:")
preview_query = """
SELECT i.inventory_id, m.make_name, mo.model_name, i.year, i.in_stock
FROM inventory i
INNER JOIN make m ON i.make_id = m.make_id
INNER JOIN model mo ON i.model_id = mo.model_id
WHERE i.year = 2021;
"""
df_preview = pd.read_sql(text(preview_query), engine)
print(df_preview)

# Update
update_query = """
UPDATE inventory
SET in_stock = FALSE
WHERE year = 2021;
"""

with engine.connect() as conn:
    result = conn.execute(text(update_query))
    conn.commit()
    print(f"\n✓ Marked {result.rowcount} car(s) as out of stock")

# Verify
print("\nUpdated status:")
df_after = pd.read_sql(text(preview_query), engine)
df_after

Cars that will be marked out of stock:
   inventory_id make_name model_name  year  in_stock
0            12    toyota      camry  2021      True
1             4      ford    mustang  2021      True

✓ Marked 2 car(s) as out of stock

Updated status:


,inventory_id,make_name,model_name,year,in_stock
0,12,toyota,camry,2021,False
1,4,ford,mustang,2021,False


## Part 5: DELETE - Removing Data

Practice DELETE statements with proper WHERE clauses.

### Exercise 5.1: Delete a specific car by inventory_id

Remove the car with `inventory_id = 10` from the inventory table.

**Use RETURNING to show what was deleted.**

In [66]:
# SOLUTION - First show what will be deleted
print("Car to be deleted:")
preview_query = """
SELECT i.inventory_id, m.make_name, mo.model_name, i.year, i.price
FROM inventory i
INNER JOIN make m ON i.make_id = m.make_id
INNER JOIN model mo ON i.model_id = mo.model_id
WHERE i.inventory_id = 10;
"""
df_preview = pd.read_sql(text(preview_query), engine)
print(df_preview)

# Delete with RETURNING
delete_query = """
DELETE FROM inventory
WHERE inventory_id = 10
RETURNING inventory_id, make_id, model_id, price;
"""

with engine.connect() as conn:
    result = conn.execute(text(delete_query))
    deleted = result.fetchone()
    conn.commit()
    print(f"\n✓ Deleted car!")
    print(f"  Inventory ID: {deleted[0]}")
    print(f"  Price: ${deleted[3]}")

Car to be deleted:
   inventory_id make_name model_name  year    price
0            10    nissan     altima  2023  38000.0

✓ Deleted car!
  Inventory ID: 10
  Price: $38000.00


### Exercise 5.2: Delete all cars NOT in stock from 2021

Remove all cars from `inventory` where:
- `in_stock = FALSE`
- `year = 2021`

**Show count of deleted rows.**

In [67]:
# SOLUTION - Preview
print("Cars to be deleted (NOT in stock AND year = 2021):")
preview_query = """
SELECT i.inventory_id, m.make_name, mo.model_name, i.year, i.in_stock
FROM inventory i
INNER JOIN make m ON i.make_id = m.make_id
INNER JOIN model mo ON i.model_id = mo.model_id
WHERE i.in_stock = FALSE AND i.year = 2021;
"""
df_preview = pd.read_sql(text(preview_query), engine)
print(df_preview)
print(f"\nTotal to delete: {len(df_preview)}")

# Delete
delete_query = """
DELETE FROM inventory
WHERE in_stock = FALSE AND year = 2021;
"""

with engine.connect() as conn:
    result = conn.execute(text(delete_query))
    conn.commit()
    print(f"\n✓ Deleted {result.rowcount} car(s)")

Cars to be deleted (NOT in stock AND year = 2021):
   inventory_id make_name model_name  year  in_stock
0            12    toyota      camry  2021     False
1             4      ford    mustang  2021     False

Total to delete: 2

✓ Deleted 2 car(s)


### Exercise 5.3: Delete cars below average price

Remove all cars from `inventory` where the price is below the average price.

**Use a subquery to calculate the average.**

In [68]:
# SOLUTION - First find average
avg_query = "SELECT AVG(price) as avg_price FROM inventory;"
df_avg = pd.read_sql(text(avg_query), engine)
avg_price = df_avg['avg_price'][0]
print(f"Average price: ${avg_price:,.2f}")

# Preview cars below average
print("\nCars below average price:")
preview_query = f"""
SELECT i.inventory_id, m.make_name, mo.model_name, i.price
FROM inventory i
INNER JOIN make m ON i.make_id = m.make_id
INNER JOIN model mo ON i.model_id = mo.model_id
WHERE i.price < {avg_price}
ORDER BY i.price;
"""
df_preview = pd.read_sql(text(preview_query), engine)
print(df_preview)

# Delete using subquery
delete_query = """
DELETE FROM inventory
WHERE price < (SELECT AVG(price) FROM inventory);
"""

with engine.connect() as conn:
    result = conn.execute(text(delete_query))
    conn.commit()
    print(f"\n✓ Deleted {result.rowcount} car(s) below average price")

Average price: $42,231.25

Cars below average price:
   inventory_id   make_name model_name    price
0            15  volkswagen      jetta  23375.0
1            13       honda     accord  25075.0
2             1      toyota      camry  28500.0
3            11       mazda       cx-5  28900.0
4             3       honda     accord  32000.0
5            17       honda     accord  34000.0
6             9        audi         a4  34850.0
7            16      toyota      camry  35000.0
8             2      toyota       rav4  42000.0

✓ Deleted 9 car(s) below average price


### Exercise 5.4: Delete unused colors

Remove all colors from the `color` table that are NOT currently used in the `inventory` table.

**Hint:** Use a subquery with NOT IN or NOT EXISTS.

In [69]:
# SOLUTION - Find unused colors
print("Colors NOT currently in inventory:")
preview_query = """
SELECT c.color_id, c.color_name
FROM color c
WHERE c.color_id NOT IN (SELECT DISTINCT color_id FROM inventory WHERE color_id IS NOT NULL)
ORDER BY c.color_id;
"""
df_preview = pd.read_sql(text(preview_query), engine)
print(df_preview)

# Delete unused colors
delete_query = """
DELETE FROM color
WHERE color_id NOT IN (SELECT DISTINCT color_id FROM inventory WHERE color_id IS NOT NULL);
"""

with engine.connect() as conn:
    result = conn.execute(text(delete_query))
    conn.commit()
    print(f"\n✓ Deleted {result.rowcount} unused color(s)")

# Show remaining colors
print("\nRemaining colors:")
df_remaining = pd.read_sql(text("SELECT * FROM color ORDER BY color_id;"), engine)
df_remaining

Colors NOT currently in inventory:
   color_id color_name
0         7       Gray
1         8      Green
2         9     Orange
3        10     Purple
4        11      Brown

✓ Deleted 5 unused color(s)

Remaining colors:

✓ Deleted 5 unused color(s)

Remaining colors:


,color_id,color_name
0,1,Blue
1,2,Silver
2,3,White
3,4,Red
4,5,Yellow
5,6,Black


## Part 6: Final Inventory Summary

View the final state of the database after all modifications.

In [70]:
# Final inventory with all details
query = """
SELECT 
    i.inventory_id,
    m.make_name,
    mo.model_name,
    c.color_name,
    ft.fuel_type_name,
    i.year,
    i.price,
    i.in_stock,
    i.mileage
FROM inventory i
INNER JOIN make m ON i.make_id = m.make_id
INNER JOIN model mo ON i.model_id = mo.model_id
LEFT JOIN color c ON i.color_id = c.color_id
LEFT JOIN fuel_type ft ON i.fuel_type_id = ft.fuel_type_id
ORDER BY i.inventory_id;
"""

df_final = pd.read_sql(text(query), engine)
print(f"Final Inventory Count: {len(df_final)} cars")
print("=" * 80)
df_final

Final Inventory Count: 7 cars


,inventory_id,make_name,model_name,color_name,fuel_type_name,year,price,in_stock,mileage
0,5,chevrolet,corvette,Yellow,gasoline,2023,75000.0,False,2000
1,6,tesla,model 3,Blue,electric,2023,48000.0,True,0
2,7,bmw,x5,Black,hybrid,2022,65000.0,True,3500
3,8,mercedes,c-class,Silver,hybrid,2023,52000.0,True,1200
4,14,ford,f-150,Yellow,gasoline,2023,52000.0,True,0
5,18,ford,mustang,Red,gasoline,2023,48000.0,True,500
6,19,tesla,model 3,White,electric,2024,52000.0,True,0


In [71]:
# Summary statistics
summary_query = """
SELECT 
    COUNT(*) as total_cars,
    COUNT(CASE WHEN in_stock = TRUE THEN 1 END) as in_stock,
    COUNT(CASE WHEN in_stock = FALSE THEN 1 END) as not_in_stock,
    ROUND(AVG(price), 2) as avg_price,
    MIN(price) as min_price,
    MAX(price) as max_price,
    ROUND(AVG(mileage), 0) as avg_mileage
FROM inventory;
"""

df_summary = pd.read_sql(text(summary_query), engine)
print("Inventory Summary Statistics:")
print("=" * 80)
df_summary

Inventory Summary Statistics:


,total_cars,in_stock,not_in_stock,avg_price,min_price,max_price,avg_mileage
0,7,6,1,56000.0,48000.0,75000.0,1029.0


## Summary

**In this homework, you practiced:**

### ✓ Database Exploration
- Querying information_schema to view table structure
- Understanding normalized database design
- Working with foreign key relationships

### ✓ COPY Operations
- Loading CSV data using pandas to_sql()
- Bulk importing data into tables with foreign keys
- Verifying loaded data with JOIN queries

### ✓ INSERT Statements
- Single row INSERT
- Multiple row INSERT (batch insert)
- INSERT with RETURNING clause
- Working with foreign key constraints

### ✓ UPDATE Operations
- Converting text to lowercase using LOWER()
- Updating multiple tables independently
- Conditional updates with WHERE clauses
- Calculations in UPDATE (discounts, rounding)

### ✓ DELETE Operations
- Single row DELETE by primary key
- Conditional DELETE with multiple criteria
- DELETE with subqueries
- Removing orphaned reference data
- Using RETURNING to verify deletions

### Key Takeaways:
1. **Normalized databases** reduce redundancy and improve data integrity
2. **Foreign keys** enforce referential integrity between tables
3. **Always use WHERE clause** in UPDATE and DELETE to avoid modifying all rows
4. **Preview with SELECT** before executing UPDATE or DELETE
5. **RETURNING clause** shows exactly what was modified
6. **Bulk operations** (COPY, batch INSERT) are more efficient than single-row operations

### Best Practices Applied:
- ✓ Used SELECT to preview changes before modifying data
- ✓ Applied WHERE clauses to target specific rows
- ✓ Used RETURNING to verify modifications
- ✓ Maintained referential integrity with foreign keys
- ✓ Used functions (LOWER, ROUND) for data transformation

**Excellent work! You now have hands-on experience with all core data manipulation operations in PostgreSQL.**